In [1]:
import logging
from tqdm import tqdm

import config
import modules.metrics_mAP_aladdin as aladdin_mAP
import modules.dataloaders as data_loaders
import modules.models as models
import modules.utils as utils
import modules.metrics as metrics

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.13 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


# Logger and Folders to Save Plots

In [2]:
log_path = './mAP_&_pr_curves/epochs=150__max_obj_10__simple_model__thres_001/'

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path + 'logfile.log')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info('Simple BED Detector.\n' +
           '\tNo Sigmoid. No Softmax. Permute out of the model.\n' +
            '\tDFire and FASDD UAV and CV.\n' +
            f'\tMax Obj = {config.MAX_OBJ}.\n' +
            f'\tIOU Threshold = {config.IOU_THRESHOLD}.\n' +
            f'\tNMS IOU Threshold = {config.NMS_IOU_THRESHOLD}.\n' +
            f'\tScore Threshold = {config.SCORE_THRESHOLD}.\n' +
            '\tNMS IOU Threshold does not change too much, so it is not tested.\n')

# Datasets DFire

## DFire

In [3]:
dfire_loader = data_loaders.get_dfire_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test dataset len: 3983


## FASDD UAV

In [4]:
fasdd_uav_loader = data_loaders.get_fasdd_uav_val_loader()


TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 377
FASDD Removed due to more than 10: 156

Test FASDD UAV dataset len: 3648


## FASDD CV

In [5]:
fasdd_cv_loader = data_loaders.get_fasdd_cv_val_loader()


TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 317
FASDD Removed due to more than 10: 44

Test FASDD CV dataset len: 15523


## DFire, FASDD UAV and CV

In [6]:
full_loader = data_loaders.get_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test DFire dataset len: 3983

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 377
FASDD Removed due to more than 10: 156

Test FASDD UAV dataset len: 3648

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 317
FASDD Removed due to more than 10: 44

Test FASDD CV dataset len: 15523

Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 7631
Concatenate with FASDD CV dataset
Test dataset len: 23154


# Load Model

In [7]:
#model = models.BED_DETECTOR().to(config.DEVICE)

# No Sigmoid, no softmax and permute out of the model
model = models.SIMPLE_BED_DETECTOR().to(config.DEVICE)

### Load Weights

In [8]:
# ___________________________________________ #
# __________ 5 MAX_OBJ, 100 epochs __________ # 
# ___________________________________________ #
# weights_path = './experiments/test_02_dfire_&_fasdd_uav_cv/weights/'
# weights_file = weights_path + 'BED_detector__best_mAP=0.6352__epoch=99.pt'

# ___________________________________________ #
# __________ 10 MAX_OBJ, 200 epochs _________ # 
# ___________________________________________ #
# weights_path = './experiments/test_03_dfire_&_fasdd_uav_cv_+epochs_maxobj10/weights/'
# weights_file = weights_path + 'BED_detector__best_mAP=0.6224__epoch=119.pt'

# ___________________________________________ #
# __________  SIMPLE BED DETECTOR   _________ # 
# __________ NO SIGMOID, NO SOFTMAX _________ # 
# _________ PERMUTE OUT OF THE MODEL ________ # 
# __________ 10 MAX_OBJ, 150 epochs _________ # 
# ___________________________________________ #
weights_path = './experiments/test_20_no_sigmoid_softmax_permute_out/weights/'
weights_file = weights_path + 'BED_detector__best_mAP=0.6405__epoch=144.pt'

logger.info(f'\nWeights file: {weights_file}')

utils.load_checkpoint(
    model_path=weights_file, 
    model=model, 
    optimizer=None, 
    scheduler=None, 
    device=config.DEVICE)

Loading Model. Trained during 144 epochs


144

### Evaluation Mode

In [9]:
model.eval()

SIMPLE_BED_DETECTOR(
  (model): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
    (dropout1): Dropout2d(p=0.3, inplace=False)
    (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU()
    (dropout2): Dropout2d(p=0.3, inplace=False)
    (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv31): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn31): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu31): ReLU()
    (conv32): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=F

# Print values

In [10]:
def print_metrics(mAP_metrics):
    mAP, avg_prec, cls_prec, cls_rec = mAP_metrics
    
    mAP_str = "mAP @0.50"
    smoke = "Smoke"
    fire = "Fire"
    
    print(f'{mAP_str:<12}' + f'{mAP:.4f}')
    print('Average Precision')
    print(f'- {smoke:<10}' + f'{avg_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{avg_prec[1]:.4f}')
    print('Class Precision')
    print(f'- {smoke:<10}' + f'{cls_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_prec[1]:.4f}')  
    print('Class Recall')
    print(f'- {smoke:<10}' + f'{cls_rec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_rec[1]:.4f}')
    
    logger.info(f'{mAP_str:<12}' + f'{mAP:.4f}')
    logger.info('Average Precision')
    logger.info(f'- {smoke:<10}' + f'{avg_prec[0]:.4f}')
    logger.info(f'- {fire:<10}' + f'{avg_prec[1]:.4f}')
    logger.info('Class Precision')
    logger.info(f'- {smoke:<10}' + f'{cls_prec[0]:.4f}')
    logger.info(f'- {fire:<10}' + f'{cls_prec[1]:.4f}')  
    logger.info('Class Recall')
    logger.info(f'- {smoke:<10}' + f'{cls_rec[0]:.4f}')
    logger.info(f'- {fire:<10}' + f'{cls_rec[1]:.4f}')

# Calculate mAP DFire

In [11]:
pred_boxes_1, true_boxes_1 = aladdin_mAP.get_bboxes(
    dfire_loader,
    model)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [02:07<00:00,  2.06s/it]


In [12]:
dfire_metrics = aladdin_mAP.mAP(
    log_path=log_path + 'dfire/',
    pred_boxes=pred_boxes_1,
    true_boxes=true_boxes_1)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.19s/it]


In [13]:
logger.info('\n\nDFire Aladdin Metrics:\n')
print_metrics(dfire_metrics)

mAP @0.50   0.4329
Average Precision
- Smoke     0.4999
- Fire      0.3659
Class Precision
- Smoke     0.0118
- Fire      0.0717
Class Recall
- Smoke     0.7097
- Fire      0.4862


## Compare to Torchmetrics

In [14]:
dfire_torchmetrics = metrics.torchmetrics_mAP(
    dfire_loader,
    model)

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [02:15<00:00,  2.19s/it]


In [15]:
print(dfire_torchmetrics)
logger.info(f'\nDFire Torchmetrics: \n{dfire_torchmetrics}')

{'mAP': tensor(0.4361), 'AP': [0.5025447010993958, 0.3697100579738617], 'AR': [0.7097107172012329, 0.48617085814476013]}


# Calculate mAP FASDD UAV

In [16]:
pred_boxes_2, true_boxes_2 = aladdin_mAP.get_bboxes(
    fasdd_uav_loader,
    model)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [01:47<00:00,  1.89s/it]


In [17]:
fasdd_uav_metrics = aladdin_mAP.mAP(
    log_path=log_path + 'fasdd_uav/',
    pred_boxes=pred_boxes_2,
    true_boxes=true_boxes_2)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.08s/it]


In [18]:
logger.info('\n\nFASDD UAV Aladdin Metrics:\n')
print_metrics(fasdd_uav_metrics)

mAP @0.50   0.6214
Average Precision
- Smoke     0.6937
- Fire      0.5491
Class Precision
- Smoke     0.0159
- Fire      0.0794
Class Recall
- Smoke     0.7502
- Fire      0.5657


### Compare to Torchmetrics

In [19]:
fasdd_uav_torchmetrics = metrics.torchmetrics_mAP(
    fasdd_uav_loader,
    model)

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [01:52<00:00,  1.98s/it]


In [20]:
print(fasdd_uav_torchmetrics)
logger.info(f'\nFASDD UAV Torchmetrics: \n{fasdd_uav_torchmetrics}')

{'mAP': tensor(0.6197), 'AP': [0.6906685829162598, 0.5487518906593323], 'AR': [0.7483221292495728, 0.5656954646110535]}


# Calculate mAP FASDD CV

In [21]:
pred_boxes_3, true_boxes_3 = aladdin_mAP.get_bboxes(
    fasdd_cv_loader,
    model)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 242/242 [07:47<00:00,  1.93s/it]


In [22]:
fasdd_cv_metrics = aladdin_mAP.mAP(
    log_path=log_path + 'fasdd_cv/',
    pred_boxes=pred_boxes_3,
    true_boxes=true_boxes_3)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:32<00:00, 76.40s/it]


In [23]:
logger.info('\n\nFASDD CV Aladdin Metrics:\n')
print_metrics(fasdd_cv_metrics)

mAP @0.50   0.7270
Average Precision
- Smoke     0.7325
- Fire      0.7215
Class Precision
- Smoke     0.0157
- Fire      0.0769
Class Recall
- Smoke     0.8602
- Fire      0.7851


### Compare to Torchmetrics

In [24]:
fasdd_cv_torchmetrics = metrics.torchmetrics_mAP(
    fasdd_cv_loader,
    model)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 242/242 [08:08<00:00,  2.02s/it]


In [25]:
print(fasdd_cv_torchmetrics)
logger.info(f'\nFASDD CV Torchmetrics: \n{fasdd_cv_torchmetrics}')

{'mAP': tensor(0.7260), 'AP': [0.7317706346511841, 0.7202243804931641], 'AR': [0.8600620627403259, 0.7850919365882874]}


# Calculate mAP FULL DS

In [26]:
pred_boxes, true_boxes = aladdin_mAP.get_bboxes(
    full_loader,
    model)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [11:58<00:00,  1.99s/it]


In [27]:
full_ds_metrics = aladdin_mAP.mAP(
    log_path=log_path + 'full_ds/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [05:18<00:00, 159.13s/it]


In [28]:
logger.info('\n\nFull Dataset Aladdin Metrics:\n')
print_metrics(full_ds_metrics)

mAP @0.50   0.6663
Average Precision
- Smoke     0.6924
- Fire      0.6402
Class Precision
- Smoke     0.0151
- Fire      0.0767
Class Recall
- Smoke     0.8160
- Fire      0.7019


### Compare to Torchmetrics

In [29]:
full_ds_torchmetrics = metrics.torchmetrics_mAP(
    full_loader,
    model)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [12:13<00:00,  2.03s/it]


In [30]:
print(full_ds_torchmetrics)
logger.info(f'\nFull Dataset Torchmetrics: \n{full_ds_torchmetrics}')

{'mAP': tensor(0.6662), 'AP': [0.6918432116508484, 0.6405062675476074], 'AR': [0.8155389428138733, 0.7019383907318115]}
